In [1]:
%cd ../..

/home/work/AnacondaProjects/combinators


In [2]:
import logging

import numpy as np
import probtorch
import torch

from examples.fep_control import fep_control
from combinators.model import active
from combinators.model import compose, foldable
from combinators.inference import importance
from combinators import utils

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
target = fep_control.MountainCarStep(state_dim=2, batch_shape=(1,), trainable=False)
proposal = fep_control.RecognitionStep(state_dim=2, observation_dim=2, batch_shape=(1,), discrete_actions=False, name='MountainCarStep', trainable=True)
agent = importance.propose(target, proposal)

In [5]:
theta, graph, log_weight = active.active_inference_test(agent, 'MountainCarContinuous-v0', online_inference=False, iterations=1000)

In [6]:
theta, graph, log_weight = active.active_inference(agent, 'MountainCarContinuous-v0', lr=1e-3, episodes=10, episode_length=1000, dream=False)

04/30/2019 21:51:24 ELBO=-1.05187373e+04 at episode 0 of length 687
04/30/2019 21:51:33 ELBO=-7.62613184e+03 at episode 1 of length 1000
04/30/2019 21:51:42 ELBO=-1.56486426e+04 at episode 2 of length 103
04/30/2019 21:51:52 ELBO=-7.91659082e+03 at episode 3 of length 422
04/30/2019 21:52:01 ELBO=-5.93959326e+03 at episode 4 of length 422
04/30/2019 21:52:10 ELBO=-5.86843115e+03 at episode 5 of length 109
04/30/2019 21:52:20 ELBO=-4.39300439e+03 at episode 6 of length 333
04/30/2019 21:52:31 ELBO=-1.22113684e+03 at episode 7 of length 137
04/30/2019 21:52:40 ELBO=-5.99076611e+03 at episode 8 of length 300
04/30/2019 21:52:49 ELBO=-5.10537305e+03 at episode 9 of length 402


In [7]:
log_weight

tensor([-5105.3730], grad_fn=<AddBackward0>)

In [8]:
theta, graph, log_weight = active.active_inference_test(agent, 'MountainCarContinuous-v0', online_inference=False, iterations=1000)

In [9]:
theta

(tensor([[0.8027, 0.2563]], grad_fn=<AddBackward0>),
 tensor([[4.4315]], grad_fn=<ExpandBackward>))

In [10]:
log_weight

tensor([-1181.7379], grad_fn=<AddBackward0>)

In [11]:
torch.save(agent, 'examples/fep_control/fep_mountain_car_agent.dat')